# Retrieval Augmented Generation (RAG) & Few-Shot Prompting Medical Chat Assistant

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from datetime import date
import uvicorn
from fastapi import FastAPI, HTTPException
app = FastAPI(version="3.0.2")
class MedicalAssistant:
    def __init__(self):
        self.project_id = "Your project id"
        self.model_id = "mistralai/Mixtral-8x7B-v0.1"
        self.api_key =  "Your api key"
        self.url = "Your url"
    def get_credentials(self):
    	return {
    		"url" : self.url,
    		"apikey" :  self.api_key
    	}
    def ReferPolicy(self,query):
        loader = PyPDFLoader("Healthcare Policy Document.pdf")
        data = loader.load()
        chunk_size = 1000
        chunk_overlap= 10
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        docs = text_splitter.split_documents(data)
        model_name = "sentence-transformers/all-mpnet-base-v2"
        model_kwargs = {"device": "cpu"}
        embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
        vectorstore = FAISS.from_documents(docs, embeddings)
        params = {
                    GenParams.MAX_NEW_TOKENS: 500,
                    GenParams.MIN_NEW_TOKENS: 1,
                    GenParams.DECODING_METHOD: DecodingMethods.GREEDY
                }
        credentials = {
            'url': self.url,
            'apikey' : self.api_key
        }
        llm = Model(
            model_id= self.model_id,
            credentials=credentials,
            params=params,
            project_id=self.project_id)
        llm = WatsonxLLM(model=llm)
        chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)
        chat_history = []
        result = chain({"question": query, "chat_history": chat_history})
        ans = str(result['answer']).replace('4','ti')
        return ans
    def NotPolicy(self,query):
        today = date.today()
        today_date = today.strftime("%Y-%m-%d")
        def get_credentials():
        	return {
        		"url" : self.url,
        		"apikey" : self.api_key
        	}
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 200,
            "repetition_penalty": 1.1,
            "temperature":0.1
        }
        
        model = Model(
        	model_id =  self.model_id,
        	params = parameters,
        	credentials = get_credentials(),
        	project_id = self.project_id
        	)
        
        prompt_input = f'''
        You are a medical assistant. Your task is to respond to the query asked by the patient, providing concise answers. 
        If patient asks for appointment book it after 2 days from {today_date}.
        If you don't understand the question, instruct the patient to call the help line at phone number: 9999999999.
        
        Query: {query}
        Anser: 
        '''
        respnse = model.generate_text(prompt=prompt_input)
        return str(respnse)

    def MedicalAssitantOrchestator(self,query):
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 200,
            "repetition_penalty": 1.1,
            "temperature":0.1
        }
        model = Model(
        	model_id =  self.model_id,
        	params = parameters,
        	credentials = self.get_credentials(),
        	project_id = self.project_id
        	)
        prompt_input = f'''
        You are an medical query orchestrator. You need to understand patient query and answer accurately in 2 answers only "Refer Policy" or "Not a Policy". Answer the Latest_Query'
    
        Query: what are the documents required for the claim?
        Answer: 'Refer Policy'
    
        Query: what is the claim process?
        Answer: 'Refer Policy'
    
        Query: what are the documents required for the claim?
        Answer: 'Refer Policy'
    
        Query: Book my appointment for tomorrow?
        Answer: 'Not a Policy'
    
        Query: what is the Healthcare Policy Document?
        Answer: 'Refer Policy'
    
        Query: what is the helpline number?
        Answer: 'Not a Policy'
    
        Query: what is the insurance Policy?
        Answer: 'Refer Policy'
    
        Latest_Query: {query}
        Answer:
        '''
        respnse = model.generate_text(prompt=prompt_input)
        if 'Refer Policy' in respnse:
            respnse = self.ReferPolicy(query)
            return respnse
        respnse =  self.NotPolicy(query)
        return str(respnse).strip()
    
def MedicalAssistantQA(query):
    orchestrator = MedicalAssistant()
    response = orchestrator.MedicalAssitantOrchestator(query)
    print('Response: ',response.strip())
    return response


# Question and Answer Medical Assistant

In [14]:
query = "What are the documents required for the insurance claim?"
print("Query: ",query)
ans = MedicalAssistantQA(query)

Query:  What are the documents required for the insurance claim?
Response:  The documents required for the insurance claim include medical records documenting the need for the plastic surgery procedure, a physician's recommendation or referral for the surgery, any pre-authorization forms obtained prior to the surgery, and an itemized billing statement from the healthcare provider.


In [15]:
query = "Does plastic surgery covered in my insurance?"
print("Query: ",query)
ans = MedicalAssistantQA(query)

Query:  Does plastic surgery covered in my insurance?
Response:  Based on the healthcare policy document provided, Plastic surgery procedures are only covered by insurance if they are deemed medically necessary. Prior authorization from the insurance provider is required before undergoing any plastic surgery procedure. If you are considering Plastic surgery for cosmetic purposes, it is likely not covered by insurance and would require full payment by the patient. To determine if your specific Plastic surgery procedure is covered, you should contact your insurance provider and provide them with the detailed medical necessity and any supporting documentation. They will be able to confirm the coverage and provide you with the necessary steps to follow for reimbursement.


In [16]:
query = "I have claimed Rs.2,00,000 insurance coverage how much total coverage do I have left?"
print("Query: ",query)
ans = MedicalAssistantQA(query)

Query:  I have claimed Rs.2,00,000 insurance coverage how much total coverage do I have left?
Response:  Based on the information provided, you have claimed Rs. 2,00,000 out of a total coverage limit of Rs. 5,00,000 for medically necessary plastic surgery procedures. Therefore, you have Rs. 3,00,000 in insurance coverage left.


In [17]:
query = "Please book an appointment with a Dermatologist?"
print("Query: ",query)
ans = MedicalAssistantQA(query)

Query:  Please book an appointment with a Dermatologist?
Response:  Sure, I can help you with that. I have booked an appointment for you with a Dermatologist on February 27th, 2024.


In [18]:
query = "What is your Help line number?"
print("Query: ",query)
ans = MedicalAssistantQA(query)

Query:  What is your Help line number?
Response:  9999999999
